In [50]:
from importlib import reload
import folium
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import src.localization as localization
reload(localization)
from src.localization import read_data
from src.localization import get_distance_in_meters
positions, df = read_data('resources/data/LocTreino_Equipe_2.csv')
bts_positions, df_btss = read_data('resources/data/dados_BTSs.csv')
center_lat = np.mean([lat for lat in bts_positions[:, 0]])
center_lon = np.mean([lon for lon in bts_positions[:, 1]])
original_pts_map = folium.Map(location=[center_lat, center_lon],
                              zoom_start=15,
                              tiles='CartoDB dark_matter')
# Plot BTSs locations
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(original_pts_map)

# Export map
original_pts_map.save("resources/maps/original_pts_map.html")

# Show map
original_pts_map

In [51]:
from operator import itemgetter, attrgetter

instance = df.iloc[[0]]
#In this Step, we will select the bts with the least pathloss, it means that the ms is probably near
#
bts1 = list(instance[['pathBTS1','taBTS1']].values)
bts2 = list(instance[['pathBTS2','taBTS2']].values)
bts3 = list(instance[['pathBTS3','taBTS3']].values)
bts4 = list(instance[['pathBTS4','taBTS4']].values)
bts5 = list(instance[['pathBTS5','taBTS5']].values)
bts6 = list(instance[['pathBTS6','taBTS6']].values)
#
#
bts_infos = [bts1,bts2,bts3,bts4,bts5,bts6]
#bts_infos.sort(key=lambda x: x[0])

lat_instance = instance['lat'].values[0]
lon_instance = instance['lon'].values[0]
print(lat_instance)
print(lon_instance)

#some_list = [(1,2),(2),(5),(3),(4),(7),(0)]
#some_list.sort()
#some_list
#bts_infos
bts_by_pathloss = [
       [bts1[0][0],bts1[0][1],0],
       [bts2[0][0],bts2[0][1],1],
       [bts3[0][0],bts3[0][1],2],
       [bts4[0][0],bts4[0][1],3],
       [bts5[0][0],bts5[0][1],4],
       [bts6[0][0],bts6[0][1],5]]
bts_sorted = sorted(bts_by_pathloss,key=itemgetter(0))
bts_sorted

-8.076634
-34.899639


[[111.29, 1.0, 1],
 [122.556666666667, 2.0, 0],
 [126.59, 1.0, 2],
 [130.623333333333, 1.0, 3],
 [132.656666666667, 2.0, 4],
 [140.056666666667, 2.0, 5]]

In [52]:
time_advance = bts_sorted[0][1]
min_radius = (time_advance) * 550
radiusA = (time_advance + 1) * 550


bts_index = bts_sorted[0][2] #Select the nearest bts index
latA = df_btss['lat'][bts_index]
lonA = df_btss['lon'][bts_index]

print(latA)
print(lonA)


time_advance = bts_sorted[1][1]
min_radius = (time_advance) * 550
radiusB = (time_advance + 1) * 550


bts_index = bts_sorted[1][2] #Select the nearest bts index
latB = df_btss['lat'][bts_index]
lonB = df_btss['lon'][bts_index]

print(latB)
print(lonB)

time_advance = bts_sorted[2][1]
min_radius = (time_advance) * 550
radiusC = (time_advance + 1) * 550


bts_index = bts_sorted[2][2] #Select the nearest bts index
latC = df_btss['lat'][bts_index]
lonC = df_btss['lon'][bts_index]

print(latC)
print(lonC)


#distance = get_distance_in_meters(lat_bts,lon_bts,-8.06500,-34.900)

#if(distance <=max_radius and distance > min_radius):
#    print('A distância é de {} metros Está dentro do range previsto'.format(distance))
#marker = folium.Circle(location=(lat_bts,lon_bts),color='blue',weight=1,radius=distance)
#marker.add_to(original_pts_map)

icon_p = folium.Icon(color='white',icon_color='green')
marker = folium.Circle(location=(latA,lonA),color='green',weight=1,radius=radiusA)
marker.add_to(original_pts_map)

marker = folium.Circle(location=(latB,lonB),color='green',weight=1,radius=radiusB)
marker.add_to(original_pts_map)

marker = folium.Circle(location=(latC,lonC),color='green',weight=1,radius=radiusC)
marker.add_to(original_pts_map)

marker = folium.Marker(location=(lat_instance,lon_instance),popup='teste',tooltip='celular',icon=icon_p)
marker.add_to(original_pts_map)

print(radiusA)
print(radiusB)
print(radiusC)

original_pts_map

-8.075916667000001
-34.894611110999996
-8.068361111
-34.892722221999996
-8.076361111
-34.908
1100.0
1650.0
1100.0


Applying Trilateration
==

In [53]:
earthRadius = 6371
print(radiusA)
print(radiusB)
print(radiusC)

#distA = distA / 1000
#distB = distB / 1000
#distC = distC / 1000
distA = radiusA/1000
distB = radiusB/1000
distC = radiusC/1000

print(distA)
print(distB)
print(distC)

import math, numpy

xA = earthRadius * (math.cos( math.radians( latA )) * math.cos( math.radians( lonA )))
yA = earthRadius * (math.cos( math.radians( latA )) * math.sin( math.radians( lonA )))
zA = earthRadius * (math.sin(math.radians(latA)))                           

xB = earthRadius * (math.cos( math.radians( latB )) * math.cos( math.radians( lonB )))
yB = earthRadius * (math.cos( math.radians( latB )) * math.sin( math.radians( lonB )))
zB = earthRadius * (math.sin(math.radians(latB)))

xC = earthRadius * (math.cos( math.radians( latC )) * math.cos( math.radians( lonC )))
yC = earthRadius * (math.cos( math.radians( latC )) * math.sin( math.radians( lonC )))
zC = earthRadius * (math.sin(math.radians(latC)))

P1 = numpy.array([xA, yA, zA])
P2 = numpy.array([xB, yB, zB])
P3 = numpy.array([xC, yC, zC])

# A partir do algorítmo da wikipédia
#       Transforma o centro da espera P1 na origem
#       Tranforma o centro da esfera P2 no eixo x


ex = (P2 - P1) / (numpy.linalg.norm(P2 - P1))       # Vetor unitário da direção de P1 para P2
i = numpy.dot(ex, P3 - P1)                          # Magnitude do sinal da componente x, do vetor de P1 para P3 
ey = (P3 - P1 - i*ex) / (numpy.linalg.norm(P3 - P1 - i*ex))     # Vetor unitário da direção de y 
ez = numpy.cross(ex, ey)                                        # ez é o produto cartesiano entre (ex) x (ey)
 
d = numpy.linalg.norm(P2 - P1)         # distância entre os centros de P1 (0,0) e P2(d,0) 
j = numpy.dot(ey, P3 - P1)             # magnitude do sinal da componente y, do vetro de P1 para P3

x = ( pow(distA, 2) - pow(distB, 2) + pow(d, 2)) / (2*d)                          # x = ((r1^2 - r2^2 + d^2)^2) / (4d^2)
y = ((pow(distA, 2) - pow(distC, 2) + pow(i, 2) + pow(j,2)) / (2*j)) - ((i/j)*x)  # y = (r1^2 - r3^2 + i^2 +j^2)/2j - (i/j)x
z = numpy.sqrt(pow(distA, 2) - pow(x, 2) - pow(y, 2))                             # z = +-sqrt(r1^2 - x^2 - y^2)

# TriPt é um array com os pontos x,y e z em ECEF, ponto de trilateração utilizado para fazer a conversão para lat/long

triPt = P1 + x*ex + y*ey + z*ez

# Converte de ECEF para lat/long e converte em graus

lat = math.degrees(math.asin(triPt[2] / earthRadius))
lon = math.degrees(math.atan2(triPt[1], triPt[0]))

print (lat, lon)
icon_predicted = folium.Icon(color='white',icon_color='blue')
marker = folium.Marker(location=(lat,lon),popup='Predicted Point',tooltip='Ponto Predito',icon=icon_predicted)
marker.add_to(original_pts_map)

1100.0
1650.0
1100.0
1.1
1.65
1.1
-8.079371644026969 -34.90122967471589


In [54]:
original_pts_map